In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 17


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 8.08 ms, sys: 16.1 ms, total: 24.2 ms
Wall time: 23.3 ms


## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, tokenizer = tokenize_sentences(x_train, max_len)
x_val_ids, tokenizer   = tokenize_sentences(x_val, max_len)


Loading previously created Tokenizer
Loading previously created Tokenizer


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer)

Loading previously created Embedding Matrix
CPU times: user 0 ns, sys: 2.85 ms, total: 2.85 ms
Wall time: 2.34 ms


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:
%%time
# this is very quick, so we can use grid search on these models.

run_logfile = "runs2ballen17.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models  (Removed the filter sizes 64 that exceed the size of the shorter sequence)
for k_sizes in [ [8,16],  [4,8,12], [4,8,16], [8,12,16] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96, 96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal17",
                      model_dir        = "../models/CNNBal17",
                      max_len          = max_len)
            
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 8ms/step - loss: 0.6836 - accuracy: 0.6179 - val_loss: 0.5824 - val_accuracy: 0.7016
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222122/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4602 - accuracy: 0.7862 - val_loss: 0.4555 - val_accuracy: 0.7893
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222122/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.3099 - accuracy: 0.8714 - val_loss: 0.3134 - val_accuracy: 0.8675
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222122/assets
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1959 - accuracy: 0.9282 - val_loss: 0.3065 - val_accuracy: 0.8825
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222122/assets
Epoch 5/20
288/288 [=====

288/288 [==============================] - 1s 5ms/step - loss: 0.0873 - accuracy: 0.9691 - val_loss: 0.2322 - val_accuracy: 0.9173
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222231/assets
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0551 - accuracy: 0.9809 - val_loss: 0.2824 - val_accuracy: 0.9137
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0387 - accuracy: 0.9877 - val_loss: 0.2869 - val_accuracy: 0.9245
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222231/assets
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0280 - accuracy: 0.9902 - val_loss: 0.2938 - val_accuracy: 0.9191
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0196 - accuracy: 0.9940 - val_loss: 0.3674 - val_accuracy: 0.9150
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0148 - accuracy: 0.9956 - val_loss: 0.3950 - val_accuracy: 0.9069
Epoch 11

Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0132 - accuracy: 0.9963 - val_loss: 0.3892 - val_accuracy: 0.9087
Epoch 11/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0170 - accuracy: 0.9937 - val_loss: 0.3401 - val_accuracy: 0.9227
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0120 - accuracy: 0.9960 - val_loss: 0.4080 - val_accuracy: 0.9132
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0111 - accuracy: 0.9965 - val_loss: 0.4570 - val_accuracy: 0.9033
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0085 - accuracy: 0.9965 - val_loss: 0.4402 - val_accuracy: 0.9177
Epoch 15/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0060 - accuracy: 0.9983 - val_loss: 0.4036 - val_accuracy: 0.9250
Epoch 16/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0081 - accuracy: 0.9970 - val_loss: 0.4811 - val_accuracy: 0.9132

288/288 [==============================] - 1s 5ms/step - loss: 0.0090 - accuracy: 0.9968 - val_loss: 0.4406 - val_accuracy: 0.9272
Epoch 18/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0121 - accuracy: 0.9955 - val_loss: 0.3619 - val_accuracy: 0.9331
Epoch 19/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0097 - accuracy: 0.9966 - val_loss: 0.3608 - val_accuracy: 0.9394
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222447/assets
Epoch 20/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0085 - accuracy: 0.9970 - val_loss: 0.5130 - val_accuracy: 0.9173



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 6ms/step - loss: 0.6339 - accuracy: 0.6406 - val_loss: 0.6651 - val_accuracy: 0.6569
INFO:tensorflow:Assets written to: 

Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2420 - accuracy: 0.9082 - val_loss: 0.3151 - val_accuracy: 0.8639
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222630/assets
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1266 - accuracy: 0.9583 - val_loss: 0.3041 - val_accuracy: 0.8883
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222630/assets
Epoch 5/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0594 - accuracy: 0.9826 - val_loss: 0.2531 - val_accuracy: 0.9155
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222630/assets
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0338 - accuracy: 0.9888 - val_loss: 0.2129 - val_accuracy: 0.9399
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-222630/assets
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0240 - accuracy: 0.9926 - val_loss: 0.4544 - val_accurac

...

Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0101 - accuracy: 0.9972 - val_loss: 0.3503 - val_accuracy: 0.9335
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-225207/assets
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0100 - accuracy: 0.9973 - val_loss: 0.5250 - val_accuracy: 0.9042
Epoch 13/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0184 - accuracy: 0.9928 - val_loss: 0.3446 - val_accuracy: 0.9304
Epoch 14/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0108 - accuracy: 0.9963 - val_loss: 0.3937 - val_accuracy: 0.9281
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0073 - accuracy: 0.9976 - val_loss: 0.5502 - val_accuracy: 0.9069
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0088 - accuracy: 0.9970 - val_loss: 0.4973 - val_accuracy: 0.9109
Epoch 17/20
288/288 [==============================] - 2s 5ms/step 

288/288 [==============================] - 2s 5ms/step - loss: 5.9920e-04 - accuracy: 0.9999 - val_loss: 0.3942 - val_accuracy: 0.9349
Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.8219 - val_accuracy: 0.8807
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0174 - accuracy: 0.9939 - val_loss: 0.1766 - val_accuracy: 0.9629
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-225322/assets
Epoch 20/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0130 - accuracy: 0.9949 - val_loss: 0.2628 - val_accuracy: 0.9534



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 64
Adding Convolution: Kernel Size: 12, Filter Count: 64
Adding Convolution: Kernel Size: 16, Filter Count: 64
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6145 - accuracy: 0.6576 - val_loss: 0.3897 

288/288 [==============================] - 2s 6ms/step - loss: 0.3480 - accuracy: 0.8453 - val_loss: 0.2921 - val_accuracy: 0.8657
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-225513/assets
Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.1492 - accuracy: 0.9457 - val_loss: 0.2241 - val_accuracy: 0.9109
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-225513/assets
Epoch 4/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0730 - accuracy: 0.9770 - val_loss: 0.2432 - val_accuracy: 0.9286
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-225513/assets
Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0356 - accuracy: 0.9882 - val_loss: 0.1719 - val_accuracy: 0.9521
INFO:tensorflow:Assets written to: ../models/CNNBal17/210409-225513/assets
Epoch 6/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0269 - accuracy: 0.9908 - val_loss: 0.2570 - val_accuracy: 0.9245
E

288/288 [==============================] - 2s 6ms/step - loss: 0.0109 - accuracy: 0.9966 - val_loss: 0.4178 - val_accuracy: 0.8960
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0224 - accuracy: 0.9916 - val_loss: 0.2797 - val_accuracy: 0.9290
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0117 - accuracy: 0.9960 - val_loss: 0.3253 - val_accuracy: 0.9326
Epoch 10/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0102 - accuracy: 0.9960 - val_loss: 0.5091 - val_accuracy: 0.8987
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0099 - accuracy: 0.9966 - val_loss: 0.3517 - val_accuracy: 0.9367
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0083 - accuracy: 0.9973 - val_loss: 0.5300 - val_accuracy: 0.9155
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0034 - accuracy: 0.9991 - val_loss: 0.3469 - val_accuracy: 0.9458
INFO:tensorfl

288/288 [==============================] - 2s 6ms/step - loss: 0.0047 - accuracy: 0.9980 - val_loss: 0.5049 - val_accuracy: 0.9137
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0082 - accuracy: 0.9976 - val_loss: 0.5736 - val_accuracy: 0.9033
Epoch 16/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0199 - accuracy: 0.9947 - val_loss: 0.2722 - val_accuracy: 0.9444
Epoch 17/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0224 - accuracy: 0.9912 - val_loss: 0.3141 - val_accuracy: 0.9376
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0054 - accuracy: 0.9982 - val_loss: 0.2876 - val_accuracy: 0.9453
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0062 - accuracy: 0.9982 - val_loss: 1.1253 - val_accuracy: 0.8137
Epoch 20/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0096 - accuracy: 0.9964 - val_loss: 0.3448 - val_accuracy: 0.9381



########





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/